.

## Average Values

Create a table filled with average values for heuristics of each class.


### Labels

#### RFI

* 1 : Unclipped RFI/Noise
* 2 : Wide-band, long-duration in time clipped RFI (2016+)
* 3 : Wide-band, short-duration in time clipped RFI (2016+)
* 4 : Wide-band, short duration clipped RFI (2015)
* 5 : Sharp bandpass transition
* 6 : Wide-band, bursty clipped RFI (2015)
* 7 : Error in spectra captue or replacement
* 8 : Systematic int/float overflow

#### Astronomical

* 0 : Interesting, follow up
* 9 : Known Pulsar

In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Preamble

Read in the dataframe and prepare the features for use in scikit-learn.

In [197]:
pd.__version__

u'0.22.0'

In [198]:
df = pd.read_pickle('featureDataframe1.pkl')

In [199]:
df.describe()

,Beam,Buffer,MJDstart,bestDM,bestSNR,BinFactor,Events,DMmax,DMmin,DMmean,...,segGaussianTestsdpearson70,segGaussianTestsdpearson71,segGaussianTestsswasum,segGaussianTestslillieforsmaxp,segGaussianTestsdpearsonpsum,segGaussianTestsswpsum,segGaussianTestslfDmin,segGaussianTestslillieforssum,segGaussianTestsdpearsonomnisum,segGaussianTestslillieforsmaxD
count,125439.000000,125439.000000,125439.000000,125439.000000,125439.000000,125439.000000,1.254390e+05,125439.000000,125439.000000,125439.000000,...,125439.000000,1.254390e+05,1.254390e+05,1.254390e+05,1.254370e+05,115207.000000,125439.000000,125439.000000,125437.000000,125439.000000
mean,3.531262,216.217652,57429.620744,1349.748866,12.915690,14.184632,6.864074e+03,2373.811865,475.119150,1446.408882,...,864.919892,1.150488e-01,1.088656e+00,1.406589e-01,8.653546e-01,4.941243,0.179698,0.832779,6833.509996,0.378509
std,2.090338,282.191040,236.046434,2608.874241,60.700000,19.310866,4.732979e+04,3724.165909,1376.271409,2404.880171,...,3205.766248,2.378982e-01,2.264931e+00,2.864823e-01,1.221021e+00,2.439075,0.158045,1.809416,11713.358846,0.173132
min,0.000000,1.000000,57197.378446,0.000000,6.001111,1.000000,1.000000e+00,3.000000,0.000000,3.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.056119,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,57275.185900,9.000000,10.541907,2.000000,6.000000e+00,12.000000,5.000000,9.310345,...,5.647462,2.868619e-289,1.341274e-19,3.440451e-63,3.363013e-22,2.671830,0.060938,0.000000,152.511213,0.235100
50%,3.000000,113.000000,57324.388785,16.000000,11.370401,4.000000,5.200000e+01,38.000000,8.000000,19.400000,...,37.735010,5.455059e-09,2.442955e-07,1.150597e-13,6.362400e-02,5.725820,0.118992,0.000000,715.107481,0.499079
75%,6.000000,301.000000,57414.833449,1236.000000,12.855733,16.000000,7.440000e+02,2559.000000,155.000000,1718.334871,...,1320.036946,5.772005e-02,1.000000e+00,6.010669e-02,1.545693e+00,7.129980,0.274827,0.000000,11786.844023,0.520505
max,6.000000,2025.000000,58098.268715,10039.000000,20954.304688,64.000000,2.473715e+06,57287.182376,10039.000000,10039.000000,...,499945.248599,9.999658e-01,8.000000e+00,1.344629e+00,6.518352e+00,8.000000,1.000000,8.000000,754758.351811,0.841315


In [200]:
df.shape

(125439, 489)

#### Re-label Mis-labelled Type 8 Events

In [201]:
reLabel = [('Beam4_fb_D20150821T195709.fil', 202),
           ('Beam0_fb_D20150906T194609.fil', 275),
           ('Beam4_fb_D20150909T235709.fil', 467),
           ('Beam6_fb_D20150925T221909.fil', 3),
           ('Beam1_fb_D20151215T221703.fil', 377),
           ('Beam1_fb_D20151215T221703.fil', 595),
           ('Beam1_fb_D20151216T231103.fil', 50),
           ('Beam1_fb_D20151217T050412.fil', 196),
           ('Beam1_fb_D20151217T050412.fil', 197),
           ('Beam1_fb_D20151217T050412.fil', 202),
           ('Beam0_fb_D20160824T104004.fil', 1392),
           ('Beam0_fb_D20160825T003010.fil', 3),
           ('Beam0_fb_D20160824T104004.fil', 1392),
           ('Beam0_fb_D20160825T003010.fil', 3),
           ('Beam0_fb_D20170310T020409.fil', 6),
           ('Beam0_fb_D20170310T020409.fil', 8),
           ('Beam0_fb_D20170310T020409.fil', 9),
           ('Beam0_fb_D20170310T020409.fil', 11),
           ('Beam0_fb_D20170310T020409.fil', 14),
           ('Beam0_fb_D20170310T020409.fil', 15),
           ('Beam0_fb_D20170310T020409.fil', 16),
           ('Beam0_fb_D20170310T020409.fil', 17),
           ('Beam0_fb_D20170310T020409.fil', 19),
           ('Beam0_fb_D20170310T020409.fil', 21),
           ('Beam0_fb_D20170310T020409.fil', 25),
           ('Beam1_fb_D20170310T020409.fil', 4),
           ('Beam4_fb_D20170310T020409.fil', 7),
           ('Beam5_fb_D20170310T020409.fil', 1),
           ('Beam5_fb_D20170310T020409.fil', 2),
           ('Beam6_fb_D20170311T184903.fil', 2),
           ('Beam1_fb_D20170312T182504.fil', 2),
           ('Beam2_fb_D20170325T011509.fil', 1),
           ('Beam1_fb_D20170310T020409.fil', 3),
           ('Beam0_fb_D20170311T184903.fil', 2),
           ('Beam1_fb_D20170311T184903.fil', 1),
           ('Beam5_fb_D20170312T182504.fil', 1),
           ('Beam1_fb_D20170310T020409.fil', 2),
           ('Beam6_fb_D20170311T184903.fil', 3),
           ('Beam4_fb_D20170312T182504.fil', 10),
           ('Beam0_fb_D20170310T020409.fil', 2),
           ('Beam4_fb_D20170310T020409.fil', 4),
           ('Beam0_fb_D20170310T020409.fil', 1),
           ('Beam0_fb_D20170310T020409.fil', 3),
           ('Beam0_fb_D20170310T020409.fil', 4),
           ('Beam0_fb_D20170310T020409.fil', 5),
           ('Beam0_fb_D20170310T020409.fil', 7),
           ('Beam0_fb_D20170310T020409.fil', 10),
           ('Beam0_fb_D20170310T020409.fil', 12),
           ('Beam0_fb_D20170310T020409.fil', 13),
           ('Beam0_fb_D20170310T020409.fil', 18),
           ('Beam0_fb_D20170310T020409.fil', 20),
           ('Beam0_fb_D20170310T020409.fil', 22),
           ('Beam0_fb_D20170310T020409.fil', 23),
           ('Beam0_fb_D20170310T020409.fil', 24),
           ('Beam1_fb_D20170310T020409.fil', 1),
           ('Beam4_fb_D20170310T020409.fil', 3),
           ('Beam4_fb_D20170310T020409.fil', 5),
           ('Beam4_fb_D20170310T020409.fil', 6),
           ('Beam0_fb_D20170311T184903.fil', 1)]

In [202]:
# Relabel mis-labelled events to overflow (label 8)
for fil,buf in reLabel:
    #print fil, buf
    df.loc[(df['filterbank']==fil) & (df['Buffer']==buf), 'Label'] = 8

In [203]:
# array = df.values.astype('float32')
# args = np.argwhere(array>1e+38)
# idx = np.hsplit(args,2)
# idx = idx.flatten()
# df.drop(index=idx, inplace=True)

# print df.shape

### Compute MJD Features

We don't care about the absolute MJDs, but the relative offset within the dataset.

In [204]:
df['relMJDmax'] = df['MJDmax'] - df['MJDmin']
df['relMJDmean'] = df['MJDmean'] - df['MJDmin']
df['relMJDmedian'] = df['MJDmedian'] - df['MJDmin']

### Drop extremely large values (overflows)

In [205]:
# df.reset_index(drop=True, inplace=True)

# #Create a dataframe with only float values, convert to a numpy array:
# floatdf = df.drop(['Beam', 'Buffer', 'TSID', 'Label', 'datfile', 'predictLabel', 'filterbank', 'MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)
# fullarr = floatdf.values.astype('float32')

# #Find row indices where values are extremely large:
# overflowargs = np.argwhere(fullarr > 1e+7)
# overflowidx = np.hsplit(overflowargs,2)[0]
# overflowidx = np.unique(overflowidx)
# overflowidx.flatten()
# print overflowidx.shape

# #Put overflows in label 8:
# for i in overflowidx:
#     df.ix[i, 'Label'] = 8
#     #print i

# #Drop overflow values:
# fullar = df.drop(overflowidx)

# fullarr = floatdf.values.astype('float32')
# print np.amax(fullarr)

### Relabel overflows as class 8 and split dataframe into labelled and unlabelled

In [206]:
for row in df.iterrows():
    idx, data = row
    if data['ofCount'] > 0.: 
        df.ix[idx, 'Label'] = 8
        #print idx

/home/inigo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [207]:
labelledDf = df[df['Label'] > -1]
labelledDf = labelledDf[labelledDf['Label'] != 8]

#Merge classes 9 and 0:
labelledDf.loc[labelledDf['Label'] == 0, 'Label'] = 9

unlabelledDf = df[df['Label'] < 0.5]

In [208]:
for col in labelledDf:
    if labelledDf[col].dtype == object:
        print col, labelledDf[col].min(), labelledDf[col].max() 
    else: 
        print col, labelledDf[col].min(), labelledDf[col].max(), labelledDf[col].std()

datfile Beam0_dm_D20150624T020452.dat Beam6_dm_D20171128T020111.dat
Beam 0 6 2.30480705035
TSID D20150624T020452 D20171211T000411
Buffer 1 1997 219.335872564
MJDstart 57197.3784462 58098.2660185 187.59829644
bestDM 0.0 10039.0 3088.02311124
bestSNR 6.00170373917 89.5587844849 6.19954793862
BinFactor 1 64 22.2342099224
Events 1 1572507 51105.5824555
DMmax 3.0 10039.0 4066.69172431
DMmin 0.0 10037.0 1861.48685505
DMmean 3.0 10037.0 2871.32381282
DMmedian 3.0 10037.0 2982.41323116
DMstd 0.0 4672.49363237 957.301494983
SNRmean 6.00170373917 18.0820162533 1.18678540755
SNRmedian 6.00170373917 13.9969091415 0.920688735303
SNRstd 0.00162585682826 10.2512000471 1.08101724943
MJDmax 57197.3784884 58098.2660869 187.598293466
MJDmin 57197.3784466 58098.2660822 187.59829424
MJDstd 0.0 6.21957061339e-05 9.2407113229e-06
MJDmean 57197.3784801 58098.2660844 187.59829381
MJDmedian 57197.3784884 58098.2660845 187.598293802
Label 1 9 2.3200285664
predictLabel -1 -1 0.0
filterbank Beam0_fb_D20150626T0341

windDedispTimeStatsmax8 -18.0009460449 263.757293701 29.1429865205
windDedispTimeStatsmax9 -24.9164276123 280.122039795 29.5317011666
windDedispTimeStatsmax10 -34.4005432129 339.872833252 29.7466593682
windDedispTimeStatsmax11 -42.527557373 288.934265137 28.6882613433
windDedispTimeStatsmax12 -45.3088798523 345.369506836 28.7872856614
windDedispTimeStatsmax13 -13.9830856323 261.715118408 28.5895646638
windDedispTimeStatsmax14 -67.962209656 291.077392578 30.5932070926
windDedispTimeStatsmax15 -33.9841842651 524.930358887 33.5138956912
windDedispTimeStatsminsnr -inf 103772.112864 nan
windDedispTimeStatsmaxsnr -96519.8394717 inf nan
windDedispTimeStatsminmean -inf 26356748.0 nan
windDedispTimeStatsmaxmin -557045.0 inf nan
windDedispTimeStatsmeansum -inf inf nan
windDedispTimeStatssnr0 -9.09977945273 17.40222443 1.65600706833
windDedispTimeStatssnr1 -7.00250746568 13.5115991112 1.53453347819
windDedispTimeStatssnr2 -8.063559005 13.3087151186 1.59682952573
windDedispTimeStatssnr3 -10.149553

pixelMean_12_1 -1.61418175697 1.47621881962 0.464494162248
pixelMax_12_2 -0.302750349045 9.59025859833 1.13502295915
pixelMin_12_2 -7.9675693512 0.660808503628 2.04285787656
pixelMean_12_2 -2.88510695991 1.50495934486 0.596543786698
pixelMax_12_3 0.0 9.62121963501 1.20555924398
pixelMin_12_3 -7.96991920471 0.731831848621 1.34702747976
pixelMean_12_3 -1.35663306713 1.54404914379 0.484731110195
pixelMax_13_0 -0.731399297714 6.05144119263 1.26681690792
pixelMin_13_0 -9.88700008392 0.977541029453 0.794296974707
pixelMean_13_0 -2.00195622444 1.89767599106 0.715217020074
pixelMax_13_1 -0.175174236298 9.21577453613 0.944494293119
pixelMin_13_1 -8.09283924103 0.186100065708 1.59144370052
pixelMean_13_1 -1.62378275394 1.48269438744 0.465172786439
pixelMax_13_2 -0.299349009991 9.13405895233 1.12280138336
pixelMin_13_2 -7.97186326981 0.604129254818 2.0421652517
pixelMean_13_2 -2.6408290863 1.51385295391 0.595525480293
pixelMax_13_3 0.0 10.6272735596 1.19305858786
pixelMin_13_3 -7.95561265945 0.72

**Apply Direct Data Cleaning Filters**

In [209]:
# # Check that data during March 2017 has been removed
# for row in labelledDf.iterrows():
#     idx, data = row
#     if (data['globalTimeStatsmean'] < -3e3) or (data['globalTimeStatsmin'] < -3e3): 
#         print idx

# for row in unlabelledDf.iterrows():
#     idx, data = row
#     if (data['globalTimeStatsmean'] < -3e3) or (data['globalTimeStatsmin'] < -3e3): 
#         print idx
        
# Check that all events which overflow have been dropped
count = 0 
for row in labelledDf.iterrows():
    idx, data = row
    if data['ofCount'] > 0.: 
        #print idx
        count += 1
print count
        
count = 0 
for row in unlabelledDf.iterrows():
    idx, data = row
    if data['ofCount'] > 0.: 
        #print idx
        count += 1
print count        

0
0


In [210]:
#(n timeseries, m features)
print labelledDf.shape
print df.shape

(12398, 492)
(125439, 492)


**Select out the labels**

In [211]:
labels = labelledDf['Label'].values

**Drop the meta-information columns**

In [212]:
labelledDf.columns #list column labels

Index([u'datfile', u'Beam', u'TSID', u'Buffer', u'MJDstart', u'bestDM',
       u'bestSNR', u'BinFactor', u'Events', u'DMmax',
       ...
       u'segGaussianTestslillieforsmaxp', u'segGaussianTestsdpearsonpsum',
       u'segGaussianTestsswpsum', u'segGaussianTestslfDmin',
       u'segGaussianTestslillieforssum', u'segGaussianTestsdpearsonomnisum',
       u'segGaussianTestslillieforsmaxD', u'relMJDmax', u'relMJDmean',
       u'relMJDmedian'],
      dtype='object', length=492)

In [213]:
featuredf = labelledDf.drop(['Beam', 'Buffer', 'datfile', 'TSID', 'Label', 'predictLabel', 'filterbank', 'MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)
unlblfeaturedf = unlabelledDf.drop(['Beam', 'Buffer', 'datfile', 'TSID', 'Label', 'predictLabel', 'filterbank', 'MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)
fulldf = df.drop(['Beam', 'Buffer', 'TSID', 'Label', 'datfile', 'predictLabel', 'filterbank', 'MJDmax', 'MJDmean', 'MJDmedian', 'MJDmin'], axis=1)

In [214]:
featuredf.columns

Index([u'MJDstart', u'bestDM', u'bestSNR', u'BinFactor', u'Events', u'DMmax',
       u'DMmin', u'DMmean', u'DMmedian', u'DMstd',
       ...
       u'segGaussianTestslillieforsmaxp', u'segGaussianTestsdpearsonpsum',
       u'segGaussianTestsswpsum', u'segGaussianTestslfDmin',
       u'segGaussianTestslillieforssum', u'segGaussianTestsdpearsonomnisum',
       u'segGaussianTestslillieforsmaxD', u'relMJDmax', u'relMJDmean',
       u'relMJDmedian'],
      dtype='object', length=481)

### Pre-Processing

Prepare the features for model building: outliers (inf, NaN) need to be replaces, the features need to be scaled to zero mean, and unity variance.

In [215]:
import sklearn.preprocessing

**Convert feature dataframe to 2-D array of floats**

In [216]:
featuresArr = featuredf.values.astype('float32')
unlblfeaturesArr = unlblfeaturedf.values.astype('float32')
fullarr = fulldf.values.astype('float32')
preProcLabels = labels.copy()

**Replace NaN and inf values**

In [217]:
featuresArr[featuresArr == -np.inf] = np.nan
featuresArr[featuresArr == np.inf] = np.nan #set infinite values to nan (could u use .isinf more easily here?)
print featuresArr[featuresArr == np.nan]

unlblfeaturesArr[unlblfeaturesArr == -1. * np.inf] = np.nan
unlblfeaturesArr[unlblfeaturesArr == np.inf] = np.nan
print unlblfeaturesArr[unlblfeaturesArr == np.nan]

fullarr[fullarr == -1. * np.inf] = np.nan
fullarr[fullarr == np.inf] = np.nan
print fullarr[fullarr == np.nan]

#replace missing values with the mean along axis 0, np.nan values replaced with 'NaN' string
#imp = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(featuresArr)
#unlblimp = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(unlblfeaturesArr)

imp1 = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(featuresArr)
preProcFeats = imp1.transform(featuresArr)

imp2 = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(unlblfeaturesArr)
unlblpreProcFeats = imp2.transform(unlblfeaturesArr)

imp3 = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(fullarr)
fullfeats = imp3.transform(fullarr)

print np.argwhere(np.isfinite(preProcFeats) != True)
print np.argwhere(np.isfinite(fullfeats) != True)

print preProcFeats.shape, fullfeats.shape

[]
[]
[]
[]
[]
(12398, 481) (125439, 481)


### Replace values very close to 0 with 0

In [218]:
# args = np.isclose(fullfeats, 0.)
# fullfeats[args] = 0

# args = np.isclose(unlblpreProcFeats, 0.)
# unlblpreProcFeats[args] = 0

# args = np.isclose(preProcFeats, 0.)
# preProcFeats[args] = 0

### Remove missed overflow values (class 8)

In [219]:
# unlabelledDf = unlabelledDf.reset_index(drop=True)
# unlblfeaturedf = unlblfeaturedf.reset_index(drop=True)

# ofargs = np.argwhere(preProcFeats > 1e+38)
# ofidx = np.hsplit(ofargs,2)[0]
# ofidx = ofidx.flatten()
# preProcFeats = np.delete(preProcFeats, ofidx, axis=0)
# preProcLabels = np.delete(preProcLabels, ofidx, axis=0)
# labelledDf = labelledDf.drop(labelledDf.index[ofidx])
# featuredf = featuredf.drop(featuredf.index[ofidx])


# ofargs = np.argwhere(preProcFeats < -1e+38)
# ofidx = np.hsplit(ofargs,2)[0]
# ofidx = ofidx.flatten()
# preProcFeats = np.delete(preProcFeats, ofidx, axis=0)
# preProcLabels = np.delete(preProcLabels, ofidx, axis=0)
# labelledDf = labelledDf.drop(labelledDf.index[ofidx])
# featuredf = featuredf.drop(featuredf.index[ofidx])

# #print unlabelledDf.shape, unlabelledDf.shape
# #print unlabelledDf.index, unlabelledDf.index

# ofargs = np.argwhere(unlblpreProcFeats > 1e+38)
# ofidx = np.hsplit(ofargs,2)[0]
# ofidx = ofidx.flatten()

# unlblpreProcFeats = np.delete(unlblpreProcFeats, ofidx, axis=0)
# unlblfeaturedf = unlblfeaturedf.drop(unlabelledDf.index[ofidx])
# unlabelledDf = unlabelledDf.drop(ofidx)

# #print unlabelledDf.shape, unlblpreProcFeats.shape

# ofargs = np.argwhere(unlblpreProcFeats < -1e+38)
# ofidx = np.hsplit(ofargs,2)[0]
# ofidx = ofidx.flatten()
# unlblpreProcFeats = np.delete(unlblpreProcFeats, ofidx, axis=0)
# unlblfeaturedf = unlblfeaturedf.drop(labels = ofidx,axis=0)
# unlabelledDf = unlabelledDf.drop(ofidx)

# #print unlabelledDf.shape, unlblpreProcFeats.shape

In [220]:
print np.amax(preProcFeats)
print np.amin(preProcFeats)

2.04391e+15
-2.55175e+08


**Scale Features**

Robust scaling uses a median filter, this is better for non-Gaussian distributions

In [221]:
rScale = sklearn.preprocessing.RobustScaler().fit(fullfeats)
#print rScale.scale_
#print rScale.center_

preProcFeats = rScale.transform(preProcFeats)

print preProcFeats.shape

#print rScale.scale_
#print rScale.center_

unlblpreProcFeats = rScale.transform(unlblpreProcFeats)

print unlblpreProcFeats.shape
print preProcFeats.shape, preProcLabels.shape

(12398, 481)
(102082, 481)
(12398, 481) (12398,)


### Look for infinite outputs

In [222]:
infidx = np.argwhere(np.isfinite(preProcFeats) != True)
print np.unique(np.hsplit(infidx,2)[0]).shape

(0,)


In [223]:
#np.set_printoptions(threshold='nan')
#print preProcFeats[infidx]

infcol = np.hsplit(infidx,2)[1]
infcol = np.unique(infcol)
inffeats = featuredf.columns[infcol]
print inffeats.shape

print inffeats

(0,)
Index([], dtype='object')


#### Write important dataframes/arrays to files

In [224]:
np.save('./processed.files/labelled/featuresnew', preProcFeats)
np.save('./processed.files/labelled/labelsnew', preProcLabels)
featuredf.to_pickle('./processed.files/labelled/featureDfnew')
oldfeaturedf.to_pickle('./processed.files/labelled/oldfeaturedf')
labelledDf.to_pickle('./processed.files/labelled/labelledDfnew')

np.save('./processed.files/raw/featuresnew', unlblpreProcFeats)
unlblfeaturedf.to_pickle('./processed.files/raw/featureDfnew')
unlabelledDf.to_pickle('./processed.files/raw/unlabelledDfnew')